In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json, os

In [ ]:
models = [
    ("<path>/Experiments/pesofFixAbl_none_10_1.0_16_3_1.0_0.99_OctreeNCA2DSegmentation/", "OctreeNCA"),
    ("<path>/Experiments/peso_med_OctreeNCA2DSegmentation/", "Med-NCA"),
    ("<path>/Experiments/peso_unet_UNet2DSegmentation/", "UNet"),
    ("<path>/Experiments/peso_UNet_efficientnet-b0_0_MinUNet2DSegmentation/", "Efficientnet"),
    ("<path>/Experiments/peso_UNet_mobilenet_v2_0_MinUNet2DSegmentation/", "Mobilenetv2"),
    ("<path>/Experiments/peso_UNet_resnet18_0_MinUNet2DSegmentation/", "Resnet18"),
    ("<path>/Experiments/peso_UNet_vgg11_0_MinUNet2DSegmentation/", "VGG11"),
    ("<path>/Experiments/peso_UnetPlusPlus_resnet34_0_MinUNet2DSegmentation/", "UNet++"),
    ("<path>/Experiments/peso_vit_SegFomer2DSegmentation/", "SegFormer"),
    ("<path>/Experiments/peso_sam_Sam2DSegmentation/", "SAM"),
]

In [10]:
annnotation_list = []
annotation_dict = {}
all_files = []
for i, (model, model_name) in enumerate(models):
    score = pd.read_csv(f"{model}/eval/standard.csv", sep='\t').loc[:, "PatchwiseDiceScore/0"] * 100

    score_std = score.std()
    score = score.mean()

    eval_file = {}
    mem_allocation = None
    if os.path.exists(f"{model}/mem_allocation.json"):
        mem_allocation = round(json.load(open(f"{model}/mem_allocation.json"))["byte"] / 1024**3, 2)
    num_params = json.load(open(f"{model}/num_params.json"))["num_params"]

    eval_file["model"] = model_name
    eval_file["mem_allocation"] = mem_allocation
    eval_file["num_params"] = num_params
    eval_file["score"] = round(score, 2)
    #eval_file["score_std"] = score_std
    df = pd.DataFrame(index=[i], data=eval_file)
    all_files.append(df)
    annnotation_list.append((num_params, score, model_name))
    annotation_dict[model_name] = (num_params, score)

all_files = pd.concat(all_files)

/tmp/ipykernel_196464/4105692321.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_files = pd.concat(all_files)


In [11]:
from IPython.display import display, HTML
display(HTML(all_files.to_html()))

,model,mem_allocation,num_params,score
0,OctreeNCA,2.25,15520,86.31
1,Med-NCA,6.87,7488,84.41
2,UNet,5.49,36951425,82.79
3,Efficientnet,2.68,6251469,88.15
4,Mobilenetv2,2.47,6628945,88.20
5,Resnet18,1.56,14328209,88.00
6,VGG11,2.50,18254033,87.66
7,UNet++,3.86,26078609,88.23
8,SegFormer,0.50,3714658,86.31
9,SAM,NaN,641090608,41.12


In [12]:
#all_files.to_latex("tables/peso.tex", 
#                   index=False, 
#                   float_format="%.2f",
#                   na_rep="-",thousands=',')

In [13]:
all_files_styled = all_files.style.format(thousands=',', na_rep="-", precision=2)
all_files_styled.hide(axis="index")

model,mem_allocation,num_params,score
OctreeNCA,2.25,"15,520",86.31
Med-NCA,6.87,"7,488",84.41
UNet,5.49,"36,951,425",82.79
Efficientnet,2.68,"6,251,469",88.15
Mobilenetv2,2.47,"6,628,945",88.20
Resnet18,1.56,"14,328,209",88.00
VGG11,2.50,"18,254,033",87.66
UNet++,3.86,"26,078,609",88.23
SegFormer,0.50,"3,714,658",86.31
SAM,-,"641,090,608",41.12


In [14]:
all_files_styled.to_latex("tables/peso.tex")